In [310]:
import pandas as pd 
import geopandas as gpd
import numpy as np 
import re

In [311]:
countriesInfo = gpd.read_file('../Data/ne_50m_admin_0_countries/ne_50m_admin_0_countries.shp')

In [312]:
pibStates = pd.read_csv('../Data/Day_28Data/pib_estados.csv')

In [313]:
gdpData =  pd.read_csv('../Data/GDP_per_country_worldbank/API_NY.GDP.MKTP.CD_DS2_en_csv_v2_9865.csv', skiprows=4)

In [314]:
mapStates = gpd.read_file('../Data/dest23gw_c/dest23cw.shp', encoding='utf-8')

In [315]:
gdpData.columns = [x.lower().replace(' ', '_') for x in gdpData.columns]
mapStates.columns = [x.lower()for x in mapStates.columns]

In [316]:
pibStates = pibStates[['estado', '2021']].rename(columns={'2021': 'gdp_state', 'estado':'mexican_state'})

In [317]:
gdpData= gdpData[['country_name', 'country_code', '2021']].rename(columns={'2021': 'gdp_country'})

In [318]:
exchangeRate = pd.read_csv('../Data/Day_28Data/banxico_tipo_de_cambio_2021.csv')

In [319]:
avgExchangeRate = np.mean(exchangeRate['tipo_de_cambio'])
avgExchangeRate

np.float64(20.205254166666666)

In [320]:
pibStates['gdp_state'] = pibStates['gdp_state'].str.replace(',', '')

In [321]:
pibStates['gdp_state'] = pd.to_numeric(pibStates['gdp_state'])

In [322]:
pibStates['gdp_state_dollar'] = (pibStates['gdp_state']/avgExchangeRate)*1e6
pibStates

,mexican_state,gdp_state,gdp_state_dollar
0,Aguascalientes,318347,1.575565e+10
1,Baja California,923218,4.569198e+10
2,Baja California Sur,212813,1.053256e+10
3,Campeche,483398,2.392437e+10
4,Coahuila de Zaragoza,901801,4.463200e+10
5,Colima,145781,7.215005e+09
6,Chiapas,379227,1.876873e+10
7,Chihuahua,878625,4.348498e+10
8,Ciudad de México,3701686,1.832041e+11
9,Durango,301207,1.490736e+10


In [323]:
countriesInfo = countriesInfo[['ISO_A2', 'SOV_A3', 'NAME_EN', 'ISO_A3']]
countriesInfo.columns = [x.lower().replace(' ', '_') for x in countriesInfo.columns]


In [324]:
from forex_python.converter import CurrencyRates
c = CurrencyRates()

In [325]:
def get_rates(x):
    rates = c.get_rate('MXN', 'USD', x)
    return rates 

In [326]:
gdpData.head()

,country_name,country_code,gdp_country
0,Aruba,ABW,3.103184e+09
1,Africa Eastern and Southern,AFE,1.086772e+12
2,Afghanistan,AFG,1.426650e+10
3,Africa Western and Central,AFW,8.459930e+11
4,Angola,AGO,6.650513e+10


In [327]:
def find_closest_country(pib, countries_df):
    differences = abs(countries_df['gdp_country'] - pib)
    closest_country = countries_df.loc[differences.idxmin()]
    return closest_country['country_name'], closest_country['country_code']



In [328]:
def find_closest_country(pib, countries_df):
    # Validar que 'pib' sea numérico
    if not isinstance(pib, (int, float)):
        raise ValueError(f"El valor de 'pib' debe ser un número, pero se recibió: {type(pib)}")
    
    # Validar que 'countries_df' tenga la columna esperada
    if 'gdp_country' not in countries_df.columns or countries_df.empty:
        raise ValueError("El DataFrame 'countries_df' está vacío o no contiene la columna 'gdp_country'.")
    
    # Convertir la columna 'gdp_country' a numérica
    countries_df['gdp_country'] = pd.to_numeric(countries_df['gdp_country'], errors='coerce')
    
    # Eliminar filas con valores NaN en 'gdp_country'
    countries_df = countries_df.dropna(subset=['gdp_country'])
    
    # Validar que haya valores suficientes para comparar
    if countries_df['gdp_country'].nunique() <= 1:
        raise ValueError("No hay suficientes valores únicos en 'gdp_country' para encontrar el país más cercano.")
    
    # Calcular las diferencias absolutas y encontrar el país más cercano
    differences = abs(countries_df['gdp_country'] - pib)
    closest_country = countries_df.loc[differences.idxmin()]
    
    # Retornar el nombre y el código del país
    return closest_country['country_name'], closest_country['country_code'], closest_country['gdp_country']


In [329]:
gdpData= gdpData[gdpData['country_code']!='CSS']

In [330]:
# Asignar país a cada estado
closest_matches = []
for _, row in pibStates.iterrows():
    closest_country, country_code, country_gdp = find_closest_country(row['gdp_state_dollar'], gdpData)
    closest_matches.append({'state': row['mexican_state'], 'country': closest_country, 'country_code': country_code, 'original_gdp': row['gdp_state_dollar'], 'country_gdp': country_gdp})

# Convertir resultados en DataFrame
closest_matches_df = pd.DataFrame(closest_matches)


In [331]:
closest_matches_df

,state,country,country_code,original_gdp,country_gdp
0,Aguascalientes,Guinea,GIN,1.575565e+10,1.609182e+10
1,Baja California,Jordan,JOR,4.569198e+10,4.629610e+10
2,Baja California Sur,New Caledonia,NCL,1.053256e+10,1.007135e+10
3,Campeche,Bosnia and Herzegovina,BIH,2.392437e+10,2.367271e+10
4,Coahuila de Zaragoza,Cameroon,CMR,4.463200e+10,4.499352e+10
5,Colima,Bermuda,BMU,7.215005e+09,7.286607e+09
6,Chiapas,Botswana,BWA,1.876873e+10,1.875095e+10
7,Chihuahua,Cameroon,CMR,4.348498e+10,4.499352e+10
8,Ciudad de México,Hungary,HUN,1.832041e+11,1.821100e+11
9,Durango,Niger,NER,1.490736e+10,1.491500e+10


In [332]:
statesMatches = pd.merge(closest_matches_df, countriesInfo, left_on='country_code', right_on='iso_a3')

In [333]:
statesMatches

,state,country,country_code,original_gdp,country_gdp,iso_a2,sov_a3,name_en,iso_a3
0,Aguascalientes,Guinea,GIN,1.575565e+10,1.609182e+10,GN,GIN,Guinea,GIN
1,Baja California,Jordan,JOR,4.569198e+10,4.629610e+10,JO,JOR,Jordan,JOR
2,Baja California Sur,New Caledonia,NCL,1.053256e+10,1.007135e+10,NC,FR1,New Caledonia,NCL
3,Campeche,Bosnia and Herzegovina,BIH,2.392437e+10,2.367271e+10,BA,BIH,Bosnia and Herzegovina,BIH
4,Coahuila de Zaragoza,Cameroon,CMR,4.463200e+10,4.499352e+10,CM,CMR,Cameroon,CMR
5,Colima,Bermuda,BMU,7.215005e+09,7.286607e+09,BM,GB1,Bermuda,BMU
6,Chiapas,Botswana,BWA,1.876873e+10,1.875095e+10,BW,BWA,Botswana,BWA
7,Chihuahua,Cameroon,CMR,4.348498e+10,4.499352e+10,CM,CMR,Cameroon,CMR
8,Ciudad de México,Hungary,HUN,1.832041e+11,1.821100e+11,HU,HUN,Hungary,HUN
9,Durango,Niger,NER,1.490736e+10,1.491500e+10,NE,NER,Niger,NER


In [334]:
print(statesMatches['state'].unique())

print(mapStates['nomgeo'].unique())


['Aguascalientes' 'Baja California' 'Baja California Sur' 'Campeche'
 'Coahuila de Zaragoza' 'Colima' 'Chiapas' 'Chihuahua' 'Ciudad de México'
 'Durango' 'Guanajuato' 'Guerrero' 'Hidalgo' 'Jalisco' 'México'
 'Michoacán de Ocampo' 'Morelos' 'Nayarit' 'Nuevo León' 'Oaxaca' 'Puebla'
 'Querétaro' 'Quintana Roo' 'San Luis Potosí' 'Sinaloa' 'Sonora' 'Tabasco'
 'Tamaulipas' 'Tlaxcala' 'Veracruz de Ignacio de la Llave' 'Yucatán'
 'Zacatecas']
['Aguascalientes' 'Baja California' 'Baja California Sur' 'Campeche'
 'Coahuila de Zaragoza' 'Colima' 'Chiapas' 'Chihuahua' 'Ciudad de México'
 'Durango' 'Guanajuato' 'Guerrero' 'Hidalgo' 'Jalisco' 'México'
 'Michoacán de Ocampo' 'Morelos' 'Nayarit' 'Nuevo León' 'Oaxaca' 'Puebla'
 'Querétaro' 'Quintana Roo' 'San Luis Potosí' 'Sinaloa' 'Sonora' 'Tabasco'
 'Tamaulipas' 'Tlaxcala' 'Veracruz de Ignacio de la Llave' 'Yucatán'
 'Zacatecas']


In [335]:
statesMatches = pd.merge(statesMatches, mapStates, left_on='state', right_on='nomgeo')

In [336]:
statesMatches['iso_a2'] = statesMatches['iso_a2'].str.lower()
statesMatches['iso_a3'] = statesMatches['iso_a3'].str.lower()


In [337]:
statesMatches = gpd.GeoDataFrame(statesMatches)

In [338]:
statesMatches.to_file('../Data/Day_28Data/pib_estados_mexico_vs_countries.geojson', driver="GeoJSON")

In [95]:
#historicalRates = pd.DataFrame(pd.date_range(start='2021-01-01', end='2021-12-31', freq='D'), columns=['date'])